In [0]:
#!pip install transformers

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import transformers 
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/jigsaw-toxic-comment-train.csv')

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
def extract_embeddings(embedding):
    ##Function gets embedding from data based on the embedding model passed.
    print(f"{embedding} embedding:")
    ##Reads data file.
    df = pd.read_csv('/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/jigsaw-toxic-comment-train.csv')
    ##Loads and defines embedding tokenizer and model.
    model_class, tokenizer_class, pretrained_weights = (transformers.TFAutoModel, transformers.AutoTokenizer, embedding)
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    
    def get_features(text, step):
        ##Function gets embedded features from text, for "step" rows at a time.
        features = []
        for i in tqdm(range(0, len(text), step)):
            end = i + step
            if len(text) - i < step:
                end = len(text)
            
            tokenized = np.asarray([tokenizer.encode(x, add_special_tokens=True, max_length = 128) for x in text[i:end]])

            max_len = max([len(x) for x in tokenized])
            input_ids = pad_sequences(tokenized, maxlen = max_len, padding = 'post')
            attention_mask = np.where(input_ids != 0, 1, 0)
            input_ids = tf.constant(input_ids)
            attention_mask = tf.constant(attention_mask)

            
            last_hidden_states = model(input_ids, attention_mask=attention_mask)

            features.extend(last_hidden_states[0][:,0,:].numpy().tolist())
        return features
    ##Gets features from title, descripton and a concatenation of title and description.
    df[f'comment_text_{embedding}_features'] = get_features(df['comment_text'], 500)
    
    ##Print the size of the embedding feature vectors for debuging.
    print(f"Embedding size: {df[f'comment_text_{embedding}_features'].apply(lambda x: len(x)).unique()}")

    embedding = embedding.split('/')[-1]
    ##Save data with feature vectors to file for next steps.
    df.to_csv(f'/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/jigsaw-toxic-comment-train-{embedding}_feautres.csv', index = False)
    print(f"{embedding} has been written to file: {embedding}_feautres.csv")

In [0]:
%%time
embeddings = ['bert-base-uncased', 'airKlizz/t5-base-multi-en-wiki-news', 'airKlizz/bart-large-multi-en-wiki-news',
              'xlm-mlm-tlm-xnli15-1024', 'xlm-roberta-base']
for embedding in embeddings:
    extract_embeddings(embedding)

bert-base-uncased embedding:


  6%|▋         | 29/448 [01:20<19:35,  2.81s/it]